# baseline only mri

In [1]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = "0" 
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import hiddenlayer as hl
import torch
import torch.nn as nn
from torch.optim import SGD,Adam
import torch.utils.data as Data
from torchvision import models
from  torchvision import transforms
from  torchvision.datasets import ImageFolder
import pickle as pkl
import torchvision.models as models
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [2]:
class FineTuneResnet50(nn.Module):
    def __init__(self):
        super(FineTuneResnet50, self).__init__()
        resnet50_net_MRI = models.resnet50(pretrained=True)
        self.features_MRI = nn.Sequential(*list(resnet50_net_MRI.children())[:-1])
        self.fc_comb = nn.Sequential(
            nn.Linear(2048,256),
            nn.ReLU(),
            nn.Dropout(p = 0.5),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Dropout(p = 0.5),
            nn.Linear(128,4)
        )

    

    def forward(self, MRI):
        MRI = self.features_MRI(MRI)
        MRI = MRI.view(MRI.size(0),-1)
        output = self.fc_comb(MRI)
        return output

In [3]:
MyResnet = FineTuneResnet50()

In [4]:
MyResnet

FineTuneResnet50(
  (features_MRI): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0

In [5]:
#定义优化器
optimizer = torch.optim.Adam(MyResnet.parameters(),lr=0.00001)
loss_func = nn.CrossEntropyLoss()#损失函数

In [6]:
#记录训练过程指标
historyl = hl.History()
#使用Canves进行可视化

canvasl = hl.Canvas()

In [7]:
from torch.utils.data import DataLoader
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, root,transform_pet = None):
        super(MyDataset, self).__init__()
        MRI_PET_match_all = pkl.load(open(root,"rb"),encoding='iso-8859-1')
        MRI = []
        PET = []
        group = []
        for index,row in MRI_PET_match_all.iterrows():
            MRI.append(row['MRI_img_array'])
            PET.append(row['PET_img_array'])
            group_ = torch.tensor(row['Group'],dtype=torch.float)
            group.append(group_)
        self.MRI = MRI
        self.PET = PET
        self.group = group  
        self.transform_pet = transform_pet

    def __getitem__(self, index):
        mri =torch.from_numpy(self.MRI[index].transpose([2,0,1])).float().to(DEVICE)
        pet = torch.from_numpy(self.PET[index].transpose([2,0,1])).float().to(DEVICE)
        pet = self.transform_pet(pet)
        group = self.group[index].to(DEVICE)
        return pet,group

    def __len__(self):
        return len(self.MRI)


train_mean_mri = [4.1620684, 4.1620684, 4.1620684]
train_std_mri = [5.2131376, 5.2131376, 5.2131376]
train_mean_pet = [4.081158, 4.081158, 4.081158] 
train_std_pet = [5.1888165, 5.1888165, 5.1888165]

test_mean_mri = [4.1623616, 4.1623616, 4.1623616]
test_std_mri = [5.2136188, 5.2136188, 5.2136188]
test_mean_pet = [4.106387, 4.106387, 4.106387]
test_std_pet = [5.18535, 5.18535, 5.18535]
    
train_transform_mri = transforms.Compose([
    transforms.Normalize(train_mean_mri,train_std_mri),
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    
])

train_transform_pet = transforms.Compose([
    transforms.Normalize(train_mean_pet,train_std_pet),
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
])

test_transform_mri = transforms.Compose([
    
    transforms.Normalize(test_mean_mri,test_std_mri),
])

test_transform_pet = transforms.Compose([
   
    transforms.Normalize(test_mean_pet,test_std_pet)
])


train_data = MyDataset("/home/gc/gechang/gec_multi_fusion/data/new_label/10fold/train1.pkl",transform_pet =train_transform_pet)
test_data = MyDataset("/home/gc/gechang/gec_multi_fusion/data/new_label/10fold/test1.pkl",transform_pet =test_transform_pet)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64)


In [8]:
#对模型进行迭代训练，对所有的数据训练epoch轮
for epoch in range(140):
    train_loss_epoch = 0
    val_loss_epoch = 0
    train_corrects = 0
    val_corrects = 0
    #对训练数据的加载器进行迭代计算
    MyResnet.train().cuda()
    for step,(mri,group) in enumerate(train_loader):
        ##计算每个batch的损失
        output = MyResnet(mri)
        loss = loss_func(output,group.long())#交叉熵损失函数
        pre_lab = torch.argmax(output,1).to(DEVICE)
        optimizer.zero_grad()#每个迭代步的梯度初始化为0
        loss.backward()#损失的后向传播，计算梯度
        optimizer.step()#使用梯度进行优化
        train_loss_epoch += loss.item()*group.size(0)
        train_corrects += torch.sum(pre_lab == group.to(DEVICE).data)
    #计算一个epoch的损失和精度
    train_loss = train_loss_epoch/len(train_data.group)
    train_acc = train_corrects.double()/len(train_data.group)
    print("---------------------------------------------------")
    print("epoch:",epoch,"train_loss:",train_loss,"train_acc:",train_acc)
     #计算在验证集上的表现
    MyResnet.eval()
    for step,(mri,group) in enumerate(test_loader):
        output = MyResnet(mri)
        loss = loss_func(output,group.long())
        pre_lab = torch.argmax(output,1).to(DEVICE)
        val_loss_epoch += loss.item()*group.size(0)
        val_corrects += torch.sum(pre_lab == group.to(DEVICE).data)

    #计算一个epoch上的输出loss和acc
    val_loss = val_loss_epoch/len(test_data.group)
    val_acc = val_corrects.double()/len(test_data.group)
    print("epoch:",epoch,"val_loss:",val_loss,"val_acc:",val_acc)
    #保存每个epoch上的输出loss和acc
    historyl.log(epoch,train_loss=train_loss,val_loss = val_loss,train_acc = train_acc.item(),val_acc = val_acc.item())
    #可视化网络训练的过程
    # with canvasl:
    #     canvasl.draw_plot([historyl["train_loss"],historyl["val_loss"]])
    #     canvasl.draw_plot([historyl["train_acc"],historyl["val_acc"]])


---------------------------------------------------
epoch: 0 train_loss: 1.3896044924981872 train_acc: tensor(0.2541, device='cuda:0', dtype=torch.float64)
epoch: 0 val_loss: 1.3824457049369812 val_acc: tensor(0.3016, device='cuda:0', dtype=torch.float64)
---------------------------------------------------
epoch: 1 train_loss: 1.3717462766609516 train_acc: tensor(0.3113, device='cuda:0', dtype=torch.float64)
epoch: 1 val_loss: 1.369602882862091 val_acc: tensor(0.3250, device='cuda:0', dtype=torch.float64)
---------------------------------------------------
epoch: 2 train_loss: 1.3388827191533854 train_acc: tensor(0.3777, device='cuda:0', dtype=torch.float64)
epoch: 2 val_loss: 1.3423312187194825 val_acc: tensor(0.3578, device='cuda:0', dtype=torch.float64)
---------------------------------------------------
epoch: 3 train_loss: 1.2678374173620943 train_acc: tensor(0.4354, device='cuda:0', dtype=torch.float64)
epoch: 3 val_loss: 1.3141547083854674 val_acc: tensor(0.3469, device='cuda:0'

KeyboardInterrupt: 